In [2]:
from google.colab import drive
import pandas as pd
import matplotlib.pylab as plt
import numpy as np
!pip install miceforest
import miceforest as mf
from miceforest import ImputationKernel
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.2/58.2 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.5 MB/s eta 0:00:00


In [3]:
drive.mount('/content/drive')
phishing_dataset = pd.read_csv('/content/drive/MyDrive/dataset_cybersecurity_michelle.csv')

pd.set_option('display.max_columns', None)
phishing_dataset.head()

Mounted at /content/drive


,qty_dot_url,qty_hyphen_url,qty_underline_url,qty_slash_url,qty_questionmark_url,qty_equal_url,qty_at_url,qty_and_url,qty_exclamation_url,qty_space_url,qty_tilde_url,qty_comma_url,qty_plus_url,qty_asterisk_url,qty_hashtag_url,qty_dollar_url,qty_percent_url,qty_tld_url,length_url,qty_dot_domain,qty_hyphen_domain,qty_underline_domain,qty_slash_domain,qty_questionmark_domain,qty_equal_domain,qty_at_domain,qty_and_domain,qty_exclamation_domain,qty_space_domain,qty_tilde_domain,qty_comma_domain,qty_plus_domain,qty_asterisk_domain,qty_hashtag_domain,qty_dollar_domain,qty_percent_domain,qty_vowels_domain,domain_length,domain_in_ip,server_client_domain,qty_dot_directory,qty_hyphen_directory,qty_underline_directory,qty_slash_directory,qty_questionmark_directory,qty_equal_directory,qty_at_directory,qty_and_directory,qty_exclamation_directory,qty_space_directory,qty_tilde_directory,qty_comma_directory,qty_plus_directory,qty_asterisk_directory,qty_hashtag_directory,qty_dollar_directory,qty_percent_directory,directory_length,qty_dot_file,qty_hyphen_file,qty_underline_file,qty_slash_file,qty_questionmark_file,qty_equal_file,qty_at_file,qty_and_file,qty_exclamation_file,qty_space_file,qty_tilde_file,qty_comma_file,qty_plus_file,qty_asterisk_file,qty_hashtag_file,qty_dollar_file,qty_percent_file,file_length,qty_dot_params,qty_hyphen_params,qty_underline_params,qty_slash_params,qty_questionmark_params,qty_equal_params,qty_at_params,qty_and_params,qty_exclamation_params,qty_space_params,qty_tilde_params,qty_comma_params,qty_plus_params,qty_asterisk_params,qty_hashtag_params,qty_dollar_params,qty_percent_params,params_length,tld_present_params,qty_params,email_in_url,time_response,domain_spf,asn_ip,time_domain_activation,time_domain_expiration,qty_ip_resolved,qty_nameservers,qty_mx_servers,ttl_hostname,tls_ssl_certificate,qty_redirects,url_google_index,domain_google_index,url_shortened,phishing
0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,13,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,7,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0.245956,1,13335,1640,551,2,2,5,292,0,-1,0,0,0,1
1,2,5,4,2,0,0,0,0,0,0,0,0,0,0,0,0,0,1,329,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,24,0,0,0,5,4,2,0,0,0,0,0,0,0,0,0,0,0,0,0,305,0,5,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,302,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0.249349,1,14618,-1,-1,2,4,2,52,0,1,0,0,0,1
2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,24,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,24,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0.528680,0,13335,5355,123,2,2,0,298,1,1,0,0,0,0
3,1,1,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,1,23,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,17,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0.111804,1,36352,-1,-1,1,2,5,1792,0,0,0,0,0,1
4,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,23,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,23,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0.171760,0,-1,7865,1631,1,2,1,299,1,1,0,0,0,0


In [4]:
phishing_dataset.shape

(129698, 112)

In [5]:
phishing_dataset.columns.to_list()

['qty_dot_url',
 'qty_hyphen_url',
 'qty_underline_url',
 'qty_slash_url',
 'qty_questionmark_url',
 'qty_equal_url',
 'qty_at_url',
 'qty_and_url',
 'qty_exclamation_url',
 'qty_space_url',
 'qty_tilde_url',
 'qty_comma_url',
 'qty_plus_url',
 'qty_asterisk_url',
 'qty_hashtag_url',
 'qty_dollar_url',
 'qty_percent_url',
 'qty_tld_url',
 'length_url',
 'qty_dot_domain',
 'qty_hyphen_domain',
 'qty_underline_domain',
 'qty_slash_domain',
 'qty_questionmark_domain',
 'qty_equal_domain',
 'qty_at_domain',
 'qty_and_domain',
 'qty_exclamation_domain',
 'qty_space_domain',
 'qty_tilde_domain',
 'qty_comma_domain',
 'qty_plus_domain',
 'qty_asterisk_domain',
 'qty_hashtag_domain',
 'qty_dollar_domain',
 'qty_percent_domain',
 'qty_vowels_domain',
 'domain_length',
 'domain_in_ip',
 'server_client_domain',
 'qty_dot_directory',
 'qty_hyphen_directory',
 'qty_underline_directory',
 'qty_slash_directory',
 'qty_questionmark_directory',
 'qty_equal_directory',
 'qty_at_directory',
 'qty_and_dir

In [6]:
phishing_dataset.nunique(axis=0)
phishing_dataset.describe().apply(lambda s: s.apply(lambda x: format(x, 'f')))

,qty_dot_url,qty_hyphen_url,qty_underline_url,qty_slash_url,qty_questionmark_url,qty_equal_url,qty_at_url,qty_and_url,qty_exclamation_url,qty_space_url,qty_tilde_url,qty_comma_url,qty_plus_url,qty_asterisk_url,qty_hashtag_url,qty_dollar_url,qty_percent_url,qty_tld_url,length_url,qty_dot_domain,qty_hyphen_domain,qty_underline_domain,qty_slash_domain,qty_questionmark_domain,qty_equal_domain,qty_at_domain,qty_and_domain,qty_exclamation_domain,qty_space_domain,qty_tilde_domain,qty_comma_domain,qty_plus_domain,qty_asterisk_domain,qty_hashtag_domain,qty_dollar_domain,qty_percent_domain,qty_vowels_domain,domain_length,domain_in_ip,server_client_domain,qty_dot_directory,qty_hyphen_directory,qty_underline_directory,qty_slash_directory,qty_questionmark_directory,qty_equal_directory,qty_at_directory,qty_and_directory,qty_exclamation_directory,qty_space_directory,qty_tilde_directory,qty_comma_directory,qty_plus_directory,qty_asterisk_directory,qty_hashtag_directory,qty_dollar_directory,qty_percent_directory,directory_length,qty_dot_file,qty_hyphen_file,qty_underline_file,qty_slash_file,qty_questionmark_file,qty_equal_file,qty_at_file,qty_and_file,qty_exclamation_file,qty_space_file,qty_tilde_file,qty_comma_file,qty_plus_file,qty_asterisk_file,qty_hashtag_file,qty_dollar_file,qty_percent_file,file_length,qty_dot_params,qty_hyphen_params,qty_underline_params,qty_slash_params,qty_questionmark_params,qty_equal_params,qty_at_params,qty_and_params,qty_exclamation_params,qty_space_params,qty_tilde_params,qty_comma_params,qty_plus_params,qty_asterisk_params,qty_hashtag_params,qty_dollar_params,qty_percent_params,params_length,tld_present_params,qty_params,email_in_url,time_response,domain_spf,asn_ip,time_domain_activation,time_domain_expiration,qty_ip_resolved,qty_nameservers,qty_mx_servers,ttl_hostname,tls_ssl_certificate,qty_redirects,url_google_index,domain_google_index,url_shortened,phishing
count,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000,129698.000000
mean,2.220612,0.370044,0.132392,1.489892,0.010956,0.240459,0.025683,0.163750,0.003547,0.001234,0.003701,0.002490,0.003377,0.005389,0.000617,0.002097,0.124458,1.053933,39.097611,1.847954,0.120303,0.000617,0.000000,0.000000,0.000000,0.000008,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.439444,18.402365,0.002691,0.004194,-0.212740,-0.256164,-0.392851,0.992282,-0.460323,-0.451379,-0.456476,-0.455211,-0.458419,-0.459336,-0.456699,-0.459413,-0.458989,-0.455905,-0.460323,-

In [7]:
phishing_dataset.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129698 entries, 0 to 129697
Data columns (total 112 columns):
 #    Column                      Non-Null Count   Dtype  
---   ------                      --------------   -----  
 0    qty_dot_url                 129698 non-null  int64  
 1    qty_hyphen_url              129698 non-null  int64  
 2    qty_underline_url           129698 non-null  int64  
 3    qty_slash_url               129698 non-null  int64  
 4    qty_questionmark_url        129698 non-null  int64  
 5    qty_equal_url               129698 non-null  int64  
 6    qty_at_url                  129698 non-null  int64  
 7    qty_and_url                 129698 non-null  int64  
 8    qty_exclamation_url         129698 non-null  int64  
 9    qty_space_url               129698 non-null  int64  
 10   qty_tilde_url               129698 non-null  int64  
 11   qty_comma_url               129698 non-null  int64  
 12   qty_plus_url                129698 non-null  int64  
 13

In [8]:
find_missing = (phishing_dataset==-1).sum()

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
  print(find_missing.sort_values(ascending=False))

qty_plus_params               117020
qty_asterisk_params           117020
qty_hashtag_params            117020
qty_dollar_params             117020
qty_percent_params            117020
params_length                 117020
tld_present_params            117020
qty_params                    117020
qty_questionmark_params       117020
qty_equal_params              117020
qty_at_params                 117020
qty_and_params                117020
qty_exclamation_params        117020
qty_space_params              117020
qty_tilde_params              117020
qty_comma_params              117020
qty_slash_params              117020
qty_underline_params          117020
qty_hyphen_params             117020
qty_dot_params                117020
file_length                    59703
qty_percent_file               59703
qty_dollar_file                59703
qty_hashtag_file               59703
qty_at_file                    59703
qty_and_file                   59703
qty_exclamation_file           59703
q

In [9]:
columns_to_drop = find_missing[find_missing > 59703].index

phishing_dataset_cleaned = phishing_dataset.drop(columns=columns_to_drop)

print(phishing_dataset_cleaned.shape)

print(phishing_dataset_cleaned.info(verbose=True, show_counts=True))

(129698, 92)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129698 entries, 0 to 129697
Data columns (total 92 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   qty_dot_url                 129698 non-null  int64  
 1   qty_hyphen_url              129698 non-null  int64  
 2   qty_underline_url           129698 non-null  int64  
 3   qty_slash_url               129698 non-null  int64  
 4   qty_questionmark_url        129698 non-null  int64  
 5   qty_equal_url               129698 non-null  int64  
 6   qty_at_url                  129698 non-null  int64  
 7   qty_and_url                 129698 non-null  int64  
 8   qty_exclamation_url         129698 non-null  int64  
 9   qty_space_url               129698 non-null  int64  
 10  qty_tilde_url               129698 non-null  int64  
 11  qty_comma_url               129698 non-null  int64  
 12  qty_plus_url                129698 non-null  int64  
 13  q

In [10]:
phishing_dataset_cleaned.replace(-1, np.nan, inplace=True)
for col in phishing_dataset_cleaned.columns:
    if col != 'time_response':
        phishing_dataset_cleaned[col] = phishing_dataset_cleaned[col].astype('Int64')
print(phishing_dataset_cleaned.info(verbose=True, show_counts=True))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129698 entries, 0 to 129697
Data columns (total 92 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   qty_dot_url                 129698 non-null  Int64  
 1   qty_hyphen_url              129698 non-null  Int64  
 2   qty_underline_url           129698 non-null  Int64  
 3   qty_slash_url               129698 non-null  Int64  
 4   qty_questionmark_url        129698 non-null  Int64  
 5   qty_equal_url               129698 non-null  Int64  
 6   qty_at_url                  129698 non-null  Int64  
 7   qty_and_url                 129698 non-null  Int64  
 8   qty_exclamation_url         129698 non-null  Int64  
 9   qty_space_url               129698 non-null  Int64  
 10  qty_tilde_url               129698 non-null  Int64  
 11  qty_comma_url               129698 non-null  Int64  
 12  qty_plus_url                129698 non-null  Int64  
 13  qty_asterisk_u

In [11]:
mice_kernel = ImputationKernel(data = phishing_dataset_cleaned,
                               save_all_iterations = True,
                               random_state = 2023)

mice_kernel.mice(10, verbose=True)
phishing_dataset_imputed = mice_kernel.complete_data()
print(phishing_dataset_imputed.head())


Initialized logger with name mice 1-10
Dataset 0
1  | domain_google_index | url_google_index | qty_ip_resolved | ttl_hostname | time_response | asn_ip | qty_redirects | domain_spf | time_domain_activation | time_domain_expiration | qty_equal_directory | qty_questionmark_directory | qty_comma_directory | qty_plus_directory | qty_asterisk_directory | qty_at_directory | qty_and_directory | qty_exclamation_directory | qty_space_directory | qty_tilde_directory | qty_hyphen_file | qty_underline_file | qty_slash_file | qty_questionmark_file | qty_equal_file | qty_at_file | qty_and_file | qty_exclamation_file | qty_space_file | qty_tilde_file | qty_comma_file | qty_hashtag_directory | qty_dollar_directory | qty_percent_directory | directory_length | qty_dot_file | qty_percent_file | qty_dollar_file | qty_hashtag_file | qty_asterisk_file | qty_plus_file | file_length | qty_dot_directory | qty_hyphen_directory | qty_underline_directory | qty_slash_directory
2  | domain_google_index | url_google_

In [12]:
check_missing = phishing_dataset_imputed.isnull().sum()

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
  print(check_missing.sort_values(ascending=False))


phishing                      0
qty_dot_url                   0
qty_hyphen_url                0
qty_underline_url             0
email_in_url                  0
time_response                 0
domain_spf                    0
asn_ip                        0
time_domain_activation        0
time_domain_expiration        0
qty_ip_resolved               0
qty_nameservers               0
qty_plus_url                  0
qty_asterisk_url              0
qty_hashtag_url               0
qty_dollar_url                0
qty_percent_url               0
qty_tld_url                   0
length_url                    0
qty_dot_domain                0
qty_hyphen_domain             0
qty_underline_domain          0
qty_slash_domain              0
qty_questionmark_domain       0
qty_equal_domain              0
qty_at_domain                 0
qty_and_domain                0
qty_exclamation_domain        0
qty_space_domain              0
qty_tilde_domain              0
qty_comma_domain              0
qty_plus

In [13]:
iso_forest = IsolationForest(n_estimators=500, contamination='auto', random_state=42)


iso_forest.fit(phishing_dataset_imputed)


outliers = iso_forest.predict(phishing_dataset_imputed)

print(outliers)

outlier_count = (outliers == -1).sum()
inlier_count = (outliers == 1).sum()

outlier_count, inlier_count

[ 1 -1  1 ...  1  1  1]


(3655, 126043)

In [14]:
labels = np.where(outliers == -1, 1, 0)

X_train, X_test, y_train, y_test = train_test_split(phishing_dataset_imputed,
                                                    labels, test_size=0.2, random_state=42)

ratio = float(np.sum(labels == 0)) / np.sum(labels == 1)

xgb_model = xgb.XGBClassifier(n_estimators=100, random_state=42, scale_pos_weight=ratio, use_label_encoder=False)

xgb_model.fit(X_train, y_train)

predictions = xgb_model.predict(X_test)

print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))

[[25164    50]
 [   26   700]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     25214
           1       0.93      0.96      0.95       726

    accuracy                           1.00     25940
   macro avg       0.97      0.98      0.97     25940
weighted avg       1.00      1.00      1.00     25940



In [15]:
labels = np.where(outliers == -1, 1, 0)


X_train, X_test, y_train, y_test = train_test_split(phishing_dataset_imputed,
                                                    labels, test_size=0.2, random_state=42)

rf_model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
rf_model.fit(X_train, y_train)


predictions = rf_model.predict(X_test)

print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))


[[25198    16]
 [   79   647]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     25214
           1       0.98      0.89      0.93       726

    accuracy                           1.00     25940
   macro avg       0.99      0.95      0.96     25940
weighted avg       1.00      1.00      1.00     25940



In [17]:
labels = np.where(outliers == -1, 1, 0)

X_train, X_test, y_train, y_test = train_test_split(phishing_dataset_imputed, labels, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


model = Sequential([
    Dense(64, input_dim=X_train_scaled.shape[1], activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001),
              metrics=['accuracy', tf.keras.metrics.Recall()])

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    verbose=1,
    restore_best_weights=True
)


history = model.fit(
    X_train_scaled,
    y_train,
    epochs=75,
    batch_size=32,
    validation_split=0.2,
    verbose=1,
    callbacks=[early_stopping]
)

results = model.evaluate(X_test_scaled, y_test)
print("Test Loss, Test Accuracy, Test Recall:", results)

Epoch 1/75
2594/2594 [==============================] - 10s 4ms/step - loss: 0.0341 - accuracy: 0.9880 - recall_1: 0.8109 - val_loss: 0.0254 - val_accuracy: 0.9940 - val_recall_1: 0.9369
Epoch 2/75
2594/2594 [==============================] - 9s 3ms/step - loss: 0.0144 - accuracy: 0.9944 - recall_1: 0.8969 - val_loss: 0.0290 - val_accuracy: 0.9953 - val_recall_1: 0.9053
Epoch 3/75
2594/2594 [==============================] - 9s 3ms/step - loss: 0.0113 - accuracy: 0.9955 - recall_1: 0.9192 - val_loss: 0.0342 - val_accuracy: 0.9953 - val_recall_1: 0.8621
Epoch 4/75
2594/2594 [==============================] - 9s 3ms/step - loss: 0.0107 - accuracy: 0.9959 - recall_1: 0.9252 - val_loss: 0.0344 - val_accuracy: 0.9954 - val_recall_1: 0.8904
Epoch 5/75
2594/2594 [==============================] - 9s 3ms/step - loss: 0.0098 - accuracy: 0.9964 - recall_1: 0.9355 - val_loss: 0.0341 - val_accuracy: 0.9950 - val_recall_1: 0.8704
Epoch 6/75
2594/2594 [==============================] - 9s 3ms/step -